In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utilis as u
import importlib
import model as m 

Using TensorFlow backend.


In [2]:
importlib.reload(u)
filename = '/data_batch_1'
X_train, y_train,Y_train = u.load_data(filename, reshape=False, clipping=True)
meanX = np.mean(X_train,axis=1)
stdX = np.std(X_train,axis=1)
X_train = (X_train-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/data_batch_2'
X_val, y_val,Y_val = u.load_data(filename, reshape=False, clipping=True)
X_val = (X_val-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/test_batch'
X_test, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)
X_test = (X_test-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

data = {'X_train':X_train, 'Y_train':Y_train, 'y_train':y_train,'X_val':X_val, 'Y_val':Y_val, 'y_val':y_val}

In [24]:
importlib.reload(m)
mlp = m.MLP()
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':500, 'n_cycles':1, 'freq':10}
mlp.cyclicLearning(data, GD_params,True, 'test_batchnorm', True, True)


  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\User\Desktop\KTH\DeepL\Assignments\DD2424\Assignment3\model.py:328: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / np.sum(np.exp(x), axis=0)
c:\Users\User\Desktop\KTH\DeepL\Assignments\DD2424\Assignment3\model.py:328: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(x) / np.sum(np.exp(x), axis=0)
	 Epoch 0: train_cost = nan, val_cost = nan,  
 	 train_acc = 0.1005, val_acc = 0.0984

 10%|█         | 1/10 [00:05<00:47,  5.24s/it]	 Epoch 100: train_cost = nan, val_cost = nan,  
 	 train_acc = 0.1005, val_acc = 0.0984

 20%|██        | 2/10 [00:09<00:40,  5.09s/it]	 Epoch 200: train_cost = nan, val_cost = nan,  
 	 train_acc = 0.1005, val_acc = 0.0984

 30%|███       | 3/10 [00:14<00:34,  5.00s/it]	 Epoch 300: train_cost = nan, val_cost = nan,  
 	 train_acc = 0.1005, val_acc = 0.0984


KeyboardInterrupt: 

In [15]:
a = [1,2,3,4]
a[:-1]

[1, 2, 3]